# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

In [25]:
# Installing Theano
#! pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
! pip install --upgrade tensorflow

# Installing Keras
! pip install --upgrade keras

# Install other packages
! pip install --upgrade pip pandas numpy matplotlib sklearn

Requirement already up-to-date: tensorflow in /Users/felizola/anaconda3/envs/idp/lib/python3.6/site-packages (2.0.0)
Requirement already up-to-date: keras in /Users/felizola/anaconda3/envs/idp/lib/python3.6/site-packages (2.3.1)
Requirement already up-to-date: pip in /Users/felizola/anaconda3/envs/idp/lib/python3.6/site-packages (19.3.1)
Requirement already up-to-date: pandas in /Users/felizola/anaconda3/envs/idp/lib/python3.6/site-packages (0.25.3)
Requirement already up-to-date: numpy in /Users/felizola/anaconda3/envs/idp/lib/python3.6/site-packages (1.17.4)
Requirement already up-to-date: matplotlib in /Users/felizola/anaconda3/envs/idp/lib/python3.6/site-packages (3.1.1)
Requirement already up-to-date: sklearn in /Users/felizola/anaconda3/envs/idp/lib/python3.6/site-packages (0.0)


# Parte 1 - Preprocessamento dos Dados

### Importar as libs

In [26]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Importar o dataset

In [27]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [28]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]
print("X >>\n",X)

X >>
 [[0.0000000e+00 0.0000000e+00 6.1900000e+02 ... 1.0000000e+00
  1.0000000e+00 1.0134888e+05]
 [0.0000000e+00 1.0000000e+00 6.0800000e+02 ... 0.0000000e+00
  1.0000000e+00 1.1254258e+05]
 [0.0000000e+00 0.0000000e+00 5.0200000e+02 ... 1.0000000e+00
  0.0000000e+00 1.1393157e+05]
 ...
 [0.0000000e+00 0.0000000e+00 7.0900000e+02 ... 0.0000000e+00
  1.0000000e+00 4.2085580e+04]
 [1.0000000e+00 0.0000000e+00 7.7200000e+02 ... 1.0000000e+00
  0.0000000e+00 9.2888520e+04]
 [0.0000000e+00 0.0000000e+00 7.9200000e+02 ... 1.0000000e+00
  0.0000000e+00 3.8190780e+04]]


/Users/felizola/anaconda3/envs/idp/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/felizola/anaconda3/envs/idp/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [30]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

---
# Parte 2 -Vamos construir uma ANN!

## Importando o Keras


In [31]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [32]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the third hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Ajustando a ANN ao conjunto de treinamento

In [33]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s 112us/step - loss: 0.4839 - accuracy: 0.7949
Epoch 2/100
8000/8000 [==============================] - 1s 94us/step - loss: 0.4210 - accuracy: 0.8112
Epoch 3/100
8000/8000 [==============================] - 1s 93us/step - loss: 0.4125 - accuracy: 0.8314
Epoch 4/100
8000/8000 [==============================] - 1s 104us/step - loss: 0.4081 - accuracy: 0.8332
Epoch 5/100
8000/8000 [==============================] - 1s 115us/step - loss: 0.4052 - accuracy: 0.8342
Epoch 6/100
8000/8000 [==============================] - 1s 112us/step - loss: 0.4045 - accuracy: 0.8324
Epoch 7/100
8000/8000 [==============================] - 1s 110us/step - loss: 0.4032 - accuracy: 0.8330
Epoch 8/100
8000/8000 [==============================] - 1s 111us/step - loss: 0.4024 - accuracy: 0.8345
Epoch 9/100
8000/8000 [==============================] - 1s 110us/step - loss: 0.4017 - accuracy: 0.8340
Epoch 10/100
8000/8000 [==============================] -

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [34]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Criando uma Confusion Matrix

In [35]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1540   55]
 [ 257  148]]


# Initiate activation function experimentation

# Initialising the 2nd ANN

In [37]:
del classifier

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid', input_dim = 11))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.add(Dense(units = 3, kernel_initializer = 'uniform', activation = 'tanh'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'relu'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s 111us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 2/100
8000/8000 [==============================] - 1s 97us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 105us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 101us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 5/100
8000/8000 [==============================] - 1s 99us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 6/100
8000/8000 [==============================] - 1s 102us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 7/100
8000/8000 [==============================] - 1s 98us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 8/100
8000/8000 [==============================] - 1s 100us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 9/100
8000/8000 [==============================] - 1s 105us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 10/100
8000/8000 [==============================] - 

In [38]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

In [39]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1595    0]
 [ 405    0]]


# Initialising the 3rd ANN

In [40]:
del classifier

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid', input_dim = 11))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s 132us/step - loss: 0.5086 - accuracy: 0.7960
Epoch 2/100
8000/8000 [==============================] - 1s 113us/step - loss: 0.4454 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 113us/step - loss: 0.4339 - accuracy: 0.7965
Epoch 4/100
8000/8000 [==============================] - 1s 101us/step - loss: 0.4297 - accuracy: 0.8000
Epoch 5/100
8000/8000 [==============================] - 1s 119us/step - loss: 0.4280 - accuracy: 0.8010
Epoch 6/100
8000/8000 [==============================] - 1s 106us/step - loss: 0.4277 - accuracy: 0.8027
Epoch 7/100
8000/8000 [==============================] - 1s 109us/step - loss: 0.4275 - accuracy: 0.8026
Epoch 8/100
8000/8000 [==============================] - 1s 105us/step - loss: 0.4275 - accuracy: 0.8027
Epoch 9/100
8000/8000 [==============================] - 1s 104us/step - loss: 0.4269 - accuracy: 0.8033
Epoch 10/100
8000/8000 [==============================]

In [41]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

In [42]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1535   60]
 [ 249  156]]


---
# Parte 4 - Evaluating, Improving and Tuning the ANN

## Evaluating the ANN

In [43]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()
print("accuracies:%s\nmean:%s\nvariance:%s".format(accuracies, mean, variance))

accuracies:%s
mean:%s
variance:%s


# Improving the ANN
# Dropout Regularization to reduce overfitting if needed

In [44]:
from keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

# Tuning the ANN

In [45]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

In [ ]:
parameters = {'batch_size': [10, 24, 32],
              'epochs': [50, 100, 200],
              'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

grid_search = grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Epoch 1/50
7200/7200 [==============================] - 1s 134us/step - loss: 0.5005 - accuracy: 0.7969
Epoch 2/50
7200/7200 [==============================] - 1s 100us/step - loss: 0.4315 - accuracy: 0.7971
Epoch 3/50
7200/7200 [==============================] - 1s 109us/step - loss: 0.4257 - accuracy: 0.8096
Epoch 4/50
7200/7200 [==============================] - 1s 106us/step - loss: 0.4214 - accuracy: 0.8231
Epoch 5/50
7200/7200 [==============================] - 1s 100us/step - loss: 0.4215 - accuracy: 0.8221
Epoch 6/50
7200/7200 [==============================] - 1s 117us/step - loss: 0.4202 - accuracy: 0.8272
Epoch 7/50
7200/7200 [==============================] - 1s 103us/step - loss: 0.4200 - accuracy: 0.8271
Epoch 8/50
7200/7200 [==============================] - 1s 103us/step - loss: 0.4198 - accuracy: 0.8247
Epoch 9/50
7200/7200 [==============================] - 1s 104us/step - loss: 0.4186 - accuracy: 0.8272
Epoch 10/50
7200/7200 [==============================] - 1s 105u

# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)